#  Índice invertido e Busca Booleana 

   Essa é a resolução da primeira parte do Lab 01 da disciplina Recuperação da Informação e Busca na Web. A atividade consiste em primeiro lugar na construção de um índice invertido a partir do arquivo de texto disponibilizado.  
    Por fim, devem ser implementados algoritmos de busca booleana que utilizarão o índice invertido para responder consultas booleanas. Esses algoritmos recebem termos fornecidos pelo usuário para a realização das buscas. Três algoritmos serão exigidos:
* 1-termo: usuário passa um termo e o algoritmo retorna os documentos associados;
* Busca AND: todos os termos da consulta devem estar presentes na página;
* Busca OR: algum dos termos da consulta deve estar presente na página.


In [25]:
import pandas as pd
import numpy as np
import nltk as nl
from collections import defaultdict

    Em primeiro momento vamos conhecer os dados que consiste em um .csv de notícias do Estadão. Temos no arquivo acesso ao título da notícia, o conteúdo da notícia e, por último, o id.

In [26]:
df = pd.read_csv("noticias_estadao.csv")
df

,titulo,conteudo,idNoticia
0,PT espera 30 mil pessoas em festa na Esplanada,BRASÍLIA - Após o desgaste provocado com o lan...,1
1,Alckmin toma posse de olho no Planalto,"Reeleito em outubro, o governador tucano Geral...",2
2,Seis obstáculos e desafios do segundo mandato ...,1. Rearranjo das contas A nova equipe econôm...,3
3,Veja os desafios dos governadores que assumem ...,"No Acre, governador reeleito quer erradicar an...",4
4,PT impulsiona cerimônia de posse da Dilma nas ...,"Os perfis da presidente Dilma Rousseff, nas re...",5
5,Dilma veta projeto de Suplicy que cria linha o...,A presidente Dilma Rousseff vetou integralment...,6
6,Dilma anuncia 14 ministros e conclui reforma m...,A presidente Dilma Rousseff concluiu nesta qua...,7
7,Dilma anuncia últimos 14 ministros e fecha equipe,O Palácio do Planalto anunciou no fim da manhã...,8
8,Ex-governador critica Alckmin pela falta de ág...,O ex-governador e integrante da Executiva do P...,9
9,Veja lista completa de ministros anunciados po...,Veja a lista de ministros já anunciados pela p...,10


Agora, vamos inicializar os três algoritmos.

In [27]:
def um_termo(termo):
    pass
    #documentos

In [28]:
def busca_and(termo1, termo2):
    pass

In [29]:
def busca_or(termo1, termo2):
    pass

In [30]:
def busca(termo):
    return um_termo(termo)

In [31]:
def busca(termo1, op, termo2):
    if(op == "AND"):
        return busca_and(termo1, termo2)
    elif(op == "OR"):
        return busca_or(termo1, termo2)

In [38]:
def gera_tokens(df):
    indexes = defaultdict(set)
    for index,row in df.iterrows():
        title = nl.word_tokenize(row["titulo"])
        notice = nl.word_tokenize(row["conteudo"])
        
        for word in title:
            word = word.lower
            if word in indexes:
                indexes[word].add(row["idNoticia"])
            else:
                indexes[word] = {row["idNoticia"]}
        
        for word in notice:
            word = word.lower
            if word in indexes:
                indexes[word].add(row["idNoticia"])
            else:
                indexes[word] = {row["idNoticia"]}
    
    return indexes
                  

In [39]:
inverted_indexes = gera_tokens(df)

LookupError: 
**********************************************************************
  Resource 'tokenizers/punkt/PY3/english.pickle' not found.
  Please use the NLTK Downloader to obtain the resource:  >>>
  nltk.download()
  Searched in:
    - '/home/dandaramcs/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************

In [7]:
def sanity_check(list_ids):
    sanity_list = [1952, 4802, 1987, 6694, 5382, 1770, 2763, 1068, 5870, 2777, 1370, 2779]
    if(len(list_ids) != len(sanity_list)):
        return False
    else:
        for i in len(list_ids):
            if list_ids[i] not in sanity_list:
                return False       
        return True

In [8]:
#nl.download()

NameError: name 'nl' is not defined